In [1]:
pip install tensorflow==2.15.0

   ---------------------------------------- 0.0/300.9 MB ? eta -:--:--
   ---------------------------------------- 0.2/300.9 MB 5.9 MB/s eta 0:00:51
   ---------------------------------------- 0.6/300.9 MB 7.5 MB/s eta 0:00:41
   ---------------------------------------- 1.2/300.9 MB 9.9 MB/s eta 0:00:31
   ---------------------------------------- 2.1/300.9 MB 12.0 MB/s eta 0:00:25
   ---------------------------------------- 2.9/300.9 MB 13.0 MB/s eta 0:00:23
    --------------------------------------- 4.7/300.9 MB 17.6 MB/s eta 0:00:17
    --------------------------------------- 6.6/300.9 MB 20.9 MB/s eta 0:00:15
   - -------------------------------------- 7.6/300.9 MB 22.1 MB/s eta 0:00:14
   - -------------------------------------- 9.3/300.9 MB 22.8 MB/s eta 0:00:13
   - -------------------------------------- 11.1/300.9 MB 28.5 MB/s eta 0:00:11
   - -------------------------------------- 12.8/300.9 MB 34.4 MB/s eta 0:00:09
   - -------------------------------------- 14.3/300.9 MB 34.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.15.0 which is incompatible.


In [3]:
import numpy as np
import pandas as pd

In [5]:
import tensorflow as tf

In [7]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [17]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)


Epoch 1/100


250/250 [==============================] - 2s 2ms/step - loss: 0.5186 - accuracy: 0.7761
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4540 - accuracy: 0.8005
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4344 - accuracy: 0.8077
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4217 - accuracy: 0.8145
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4095 - accuracy: 0.8217
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3970 - accuracy: 0.8282
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3856 - accuracy: 0.8340
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3770 - accuracy: 0.8388
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3702 - accuracy: 0.8438
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3653 - acc

In [18]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1487  108]
 [ 181  224]]


0.8555